In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.saving import register_keras_serializable

2025-03-08 10:20:14.617669: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 10:20:14.621426: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 10:20:14.633900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741416614.655705    5993 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741416614.662287    5993 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
california_housing = fetch_california_housing()
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

# Splitting data into train and test.

In [3]:
X_train0, X_test, Y_train0, Y_test = train_test_split(california_housing["data"], california_housing["target"])

In [4]:
X_train1, X_validation, Y_train1, Y_validation = train_test_split(X_train0, Y_train0)

In [5]:
X_test.shape

(5160, 8)

In [6]:
X_train0.shape

(15480, 8)

In [7]:
X_train1.shape

(11610, 8)

In [8]:
X_validation.shape

(3870, 8)

# Normalizing data
using "sklearn.preprocessing.StandardScaler" function for normalizing data

In [9]:
sc = StandardScaler()
X_train_s = sc.fit_transform(X_train1)
X_validation_s = sc.fit_transform(X_validation)
X_test_s = sc.transform(X_test)

--------------------------------

In [10]:
X_train0.shape[1:]

(8,)

In [11]:
X_train_s_1, X_train_s_2 = X_train_s[:, :6], X_train_s[:, :-4]
X_validation_s_1, X_validation_s_2 = X_validation_s[:, :6], X_validation_s[:, :-4]
X_test_s_1, X_test_s_2 = X_test_s[:, :6], X_test_s[:, :-4]

# Subclass API

In [12]:
@register_keras_serializable()
class WideAndDeepANN(keras.Model):
    def __init__(self, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden_layer_1 = keras.layers.Dense(50, activation=activation)
        self.hidden_layer_2 = keras.layers.Dense(10, activation=activation)
        self.final_output = keras.layers.Dense(1)
        self.helper_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_1, input_2 = inputs
        h1_out = self.hidden_layer_1(input_1)
        h2_out = self.hidden_layer_2(h1_out)
        concat_out = keras.layers.concatenate([input_2, h2_out])
        final_out = self.final_output(concat_out)
        helper_out = self.helper_output(h2_out)
        return final_out, helper_out

In [13]:
model_f = WideAndDeepANN()

# Callback

In [21]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint("model_cb_regression_housing.keras", save_best_only=True)
early_stopping_callback = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Compile Model

In [20]:
model_f.compile(loss=["mse", "mse"],
                loss_weights=[0.8, 0.2],
                optimizer="sgd",
                metrics=[["mean_absolute_error"], ["mean_absolute_error"]])

# Fit Model

In [16]:
model_f.fit((X_train_s_1, X_train_s_2), (Y_train1, Y_train1), epochs=5,
            validation_data=((X_validation_s_1, X_validation_s_2), (Y_validation, Y_validation)),
            callbacks=[model_checkpoint_callback, early_stopping_callback])

Epoch 1/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 2.0259 - mean_absolute_error: 0.9049 - mean_absolute_error_1: 1.2863 - mse_loss: 2.8882 - val_loss: 1.1991 - val_mean_absolute_error: 0.5562 - val_mean_absolute_error_1: 0.5867 - val_mse_loss: 0.6241
Epoch 2/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.8914 - mean_absolute_error: 0.5737 - mean_absolute_error_1: 0.5900 - mse_loss: 0.6448 - val_loss: 0.5085 - val_mean_absolute_error: 0.5231 - val_mean_absolute_error_1: 0.5283 - val_mse_loss: 0.5170
Epoch 3/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5596 - mean_absolute_error: 0.5482 - mean_absolute_error_1: 0.5622 - mse_loss: 0.5751 - val_loss: 0.4680 - val_mean_absolute_error: 0.5036 - val_mean_absolute_error_1: 0.5078 - val_mse_loss: 0.4786
Epoch 4/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.5663 - mean_absolute_error: 0.5475 - mean_absolute_error_1: 0.5551 - mse_loss: 0.5767 - val_loss: 0.4784 - val_mean_absolute_error: 0.5225 - val_mean_absolute_err

In [17]:
model_f.save("housing_reg_model.keras")

In [18]:
model_f_reg = keras.models.load_model("housing_reg_model.keras")

In [19]:
model_f_reg.summary()

Model: "wide_and_deep_ann"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 50)             │           350 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            15 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 888 (3.47 KB)

 Trainable params: 886 (3.46 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [22]:
class Mycallback(keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        print("")
    def on_epoch_end(self, epoch, logs):
        print(logs["var_loss"])

In [23]:
mycb = Mycallback